In [1]:
import gym
import random



In [2]:
env = gym.make("CartPole-v1")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [3]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episodes, score))

Episode:10 Score:10.0
Episode:10 Score:33.0
Episode:10 Score:18.0
Episode:10 Score:16.0
Episode:10 Score:16.0
Episode:10 Score:21.0
Episode:10 Score:28.0
Episode:10 Score:9.0
Episode:10 Score:9.0
Episode:10 Score:12.0


In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [7]:
model = build_model(states, actions)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [9]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn
    

In [10]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-03), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

2023-04-04 23:50:07.389286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 23:50:07.393681: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 5:39 - reward: 1.0000

/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 29s 3ms/step - reward: 1.0000
113 episodes - episode_reward: 88.062 [9.000, 365.000] - loss: 1.088 - mae: 18.125 - mean_q: 36.682

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 1.0000
49 episodes - episode_reward: 204.755 [13.000, 319.000] - loss: 2.965 - mae: 39.128 - mean_q: 78.631

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 1.0000
40 episodes - episode_reward: 249.525 [119.000, 367.000] - loss: 0.673 - mae: 43.013 - mean_q: 86.449

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 1.0000
35 episodes - episode_reward: 282.714 [54.000, 415.000] - loss: 2.331 - mae: 42.861 - mean_q: 86.029

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 1.0000
done, took 143.072 seconds


In [12]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 484.000, steps: 484
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 495.000, steps: 495
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 473.000, steps: 473
Episode 25: reward: 

In [13]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 474.000, steps: 474
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
